<a href="https://colab.research.google.com/github/mostafadentist/python-ipynb/blob/main/Reliability_%26_Replacement_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np

# Component reliabilities
R = [0.9, 0.95, 0.92]

# Series system = product of reliabilities
R_series = np.prod(R)

# Parallel system = 1 - product of (1-Ri)
R_parallel = 1 - np.prod([1-r for r in R])

print("Reliability of Series System:", round(R_series,3))
print("Reliability of Parallel System:", round(R_parallel,3))

Reliability of Series System: 0.787
Reliability of Parallel System: 1.0


In [49]:
from math import comb

def k_out_of_n(n, k, p):
    reliability = sum(comb(n,i)*(p**i)*((1-p)**(n-i)) for i in range(k,n+1))
    return reliability

print("Reliability of 2-out-of-3 system (p=0.9):", round(k_out_of_n(3,2,0.9),3))

Reliability of 2-out-of-3 system (p=0.9): 0.972


In [50]:
lmbda = 0.01  # failure rate per hour
MTTF = 1/lmbda
print("Mean Time to Failure (hours):", MTTF)

# Reliability at t = 50 hours
t = 50
R_t = np.exp(-lmbda*t)
print(f"Reliability at {t} hours:", round(R_t,3))

Mean Time to Failure (hours): 100.0
Reliability at 50 hours: 0.607


In [51]:
def replacement_cost(T, cost_replacement, failure_rate):
    # T = replacement cycle time
    # failure_rate = λ
    from scipy.integrate import quad

    # Expected cost = preventive replacement + failure cost (if before T)
    def density(t): return failure_rate*np.exp(-failure_rate*t)
    prob_failure_before_T, _ = quad(density, 0, T)

    expected_cost = prob_failure_before_T*cost_replacement + (1-prob_failure_before_T)*cost_replacement
    expected_cycle = (1/failure_rate)*(1-np.exp(-failure_rate*T)) + T*np.exp(-failure_rate*T)

    return expected_cost/expected_cycle

print("Cost per unit time:", replacement_cost(100, 500, 0.01))

Cost per unit time: 5.0


In [52]:
def group_replacement(n, T, p, cost_individual, cost_group):
    # n components, group replaced every T, probability p of failure per period
    import random
    total_cost = 0
    failures = 0

    for t in range(T):
        for i in range(n):
            if random.random() < p:  # failure
                total_cost += cost_individual
                failures += 1

    total_cost += cost_group  # group replacement
    return total_cost / T

print("Average cost per period:", group_replacement(10, 5, 0.1, 50, 300))

Average cost per period: 110.0


In [53]:
purchase_cost = 10000
maintenance = [1000, 2000, 3000, 5000, 7000, 9000]

avg_costs = []
for t in range(1,len(maintenance)+1):
    total_cost = purchase_cost + sum(maintenance[:t])
    avg_cost = total_cost/t
    avg_costs.append(avg_cost)

optimal_year = np.argmin(avg_costs)+1
print("Optimal replacement year:", optimal_year)
print("Average costs per year:", avg_costs)

Optimal replacement year: 4
Average costs per year: [11000.0, 6500.0, 5333.333333333333, 5250.0, 5600.0, 6166.666666666667]
